# **Example of *Paper***

In this notebook we provide the code that can be used to obtain the results from the *paper* in Section *6* to get the first basis elements of the almost-commuting basis for generic operators $L_3$ and $L_5$ of order 3 and 5, respectively.

This implementation is done for these particular cases, and they show intermidiate steps. The complete process can be seen in the method `almost_commuting_wilson` in `dalgebra.commutators.almost_commuting`.

In [2]:
import sys
sys.path.insert(0, "..")
from dalgebra import *
%display latex

In [3]:
import re
from functools import reduce
def poly_to_latex(poly, name_u=None,name_y=None):
    r'''
        Transform a polynomial to a proper LaTeX representation.
        
        We assume the polynomial has variables `u_i`, `c_{i,j}` and `y_i` where the variables `u_i` and `y_i` are differential variables.
        The differential variables are given with `y_i^{(j)}` --  "yi_j".
        The constant variables are given with `c_{i,j}` -- "c_i_j".
        
        This method also guarantees the removal of "*" for multiplications and the "+ (-...)".
    '''
    if not isinstance(poly, str):
        poly = str(poly)
        
    def dvariable_transform(M):
        vname, weight, order = M.groups()
        weight = int(weight)
        order = int(order)
        
        if order == 0: order=r""
        elif order == 1: order="'"
        elif order == 2: order="''"
        elif order == 3: order="'''"
        else: order = r"^{(" + str(order) + r")}"
        
        if name_u != None and vname == "u": vname = name_u
        if name_y != None and vname == "y": vname = name_y
        
        return f"{vname}_\u007b{weight}\u007d{order}"
        
    patterns = list()
    
    patterns.append((r"\*", "")) # We remove the multiplication symbol
    patterns.append((r"\+ \(-([^\)]+)\)", lambda M: f"- {M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"\(-([^\)]+)\)", lambda M: f"-{M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"(\d+)/(\d+)", lambda M: f"\\frac\u007b{M.groups()[0]}\u007d\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"c_(\d+)_(\d+)", lambda M: f"c_\u007b{M.groups()[0]},{M.groups()[1]}\u007d")) # Rewritting constant variables
    ## Rewriting differential variables
    patterns.append((r"([uy])_(\d+)_(\d+)", dvariable_transform))
    patterns.append((r"z_(\d+)", lambda M: f"\\partial^\u007b{M.groups()[0]}\u007d" if int(M.groups()[0]) != 0 else ""))
    
    return reduce(lambda p, q : re.sub(q[0], q[1], p), patterns, poly)

def print_pseudo(op):
    return poly_to_latex(op[0]) + r" + \left(" + poly_to_latex(op[1]) + r"\right)\partial^{-1}"

In [4]:
from dalgebra.commutators.almost_commuting import hierarchy, recursion, almost_commuting_wilson, generic_normal
import logging
logging.getLogger("dalgebra.commutators").setLevel(logging.INFO)
I = lambda p : p.inverse_operation(0) # computes the integral of an element (or tries)
ACT = lambda op, p: op[0](**{'z': p}) + (op[1]*I(p) if (not op[1] == 0) else op[1]) # computes the action of a pseudo_operator of order -1 `op` over `p`
ACT_M = lambda M, ps: ps.__class__([sum(ACT(op, p) for (op, p) in zip(M[i], ps)) for i in range(len(M))]) # applies a matrix of p.o. of order -1 `M` over a vector of `ps`.

## Case of order 3

In [38]:
## Setting up the framework to work
R.<u_2,u_3,y_2,y_3,y_4,y_5,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u_2, u_3)
y = (1, 0, y_2, y_3, y_4, y_5)
L = sum(z[i]*u[3-i] for i in range(4))
## Showing L
LatexExpr(poly_to_latex(L))

u_{2}\partial^{1} + u_{3} + \partial^{3}

In [40]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 5): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

\tilde{P}_{0}(U,Y) \longrightarrow 1

\tilde{P}_{1}(U,Y) \longrightarrow \partial^{1}

\tilde{P}_{2}(U,Y) \longrightarrow y_{2} + \partial^{2}

\tilde{P}_{3}(U,Y) \longrightarrow y_{2}\partial^{1} + y_{3} + \partial^{3}

\tilde{P}_{4}(U,Y) \longrightarrow y_{2}\partial^{2} + y_{3}\partial^{1} + y_{4} + \partial^{4}

\tilde{P}_{5}(U,Y) \longrightarrow y_{2}\partial^{3} + y_{3}\partial^{2} + y_{4}\partial^{1} + y_{5} + \partial^{5}

#### **1. Computing the differential systems**

In [41]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [42]:
diff_systems = [tuple([bracket.coefficient_full(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [43]:
bracket = lie_brackets[1]
LatexExpr(
    r"[L_3, \tilde{P}_1(U)] = " + 
    "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
    for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)

In [44]:
print(_)

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)


In [45]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

()

##### The system for $m=2$

In [46]:
bracket = lie_brackets[2]
LatexExpr(
    r"[L_3, \tilde{P}_2(U, y_2)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
    for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)

In [47]:
print(_)

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)


In [48]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(p) + r"&= 0" for p in diff_systems[2]
    ) + "\n" + r"\end{array}\right."
)

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [49]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 3$

In [50]:
bracket = lie_brackets[3]
LatexExpr(
    r"[L_3, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(u_{2}y_{2}' - u_{2}'y_{2} - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)

In [51]:
print(_)

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(u_{2}y_{2}' - u_{2}'y_{2} - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)


In [52]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(p) + r"&= 0" for p in diff_systems[3]
    ) + "\n" + r"\end{array}\right."
)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [53]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 4$

In [54]:
bracket = lie_brackets[4]
LatexExpr(
    r"[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(u_{2}y_{3}' - u_{2}'y_{3} - u_{2}''y_{2} - u_{2}^{(4)} - 2u_{3}'y_{2} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}'y_{3} - u_{3}''y_{2} - u_{3}^{(4)} + y_{4}'''\right)

In [55]:
print(_)

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(u_{2}y_{3}' - u_{2}'y_{3} - u_{2}''y_{2} - u_{2}^{(4)} - 2u_{3}'y_{2} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}'y_{3} - u_{3}''y_{2} - u_{3}^{(4)} + y_{4}'''\right)


In [56]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(p) + r"&= 0" for p in diff_systems[4]
    ) + "\n" + r"\end{array}\right."
)

\left\{\begin{array}{rl}
	u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [57]:
print(_)

\left\{\begin{array}{rl}
	u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 5$

In [58]:
bracket = lie_brackets[5]
LatexExpr(
    r"[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} - 3u_{3}'y_{2} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(u_{2}y_{4}' - u_{2}'y_{4} - u_{2}''y_{3} - u_{2}'''y_{2} - u_{2}^{(5)} - 2u_{3}'y_{3} - 3u_{3}''y_{2} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'y_{4} - u_{3}''y_{3} - u_{3}'''y_{2} - u_{3}^{(5)} + y_{5}'''\right)

In [59]:
print(_)

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} - 3u_{3}'y_{2} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(u_{2}y_{4}' - u_{2}'y_{4} - u_{2}''y_{3} - u_{2}'''y_{2} - u_{2}^{(5)} - 2u_{3}'y_{3} - 3u_{3}''y_{2} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'y_{4} - u_{3}''y_{3} - u_{3}'''y_{2} - u_{3}^{(5)} + y_{5}'''\right)


In [60]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(p) + r"&= 0" for p in diff_systems[5]
    ) + "\n" + r"\end{array}\right."
)

\left\{\begin{array}{rl}
	u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} - 3u_{3}'y_{2} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [61]:
print(_)

\left\{\begin{array}{rl}
	u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} - 3u_{3}'y_{2} - 10u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.


#### **2. Computing the basis of $W(L_5)$**

In [62]:
W = R.weight_func([2,3,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(6)]

In [63]:
for i in range(6):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

\mathcal{T}_{0} = \{1\}

\mathcal{T}_{1} = \{\}

\mathcal{T}_{2} = \{u_{2}\}

\mathcal{T}_{3} = \{u_{2}',u_{3}\}

\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}

\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}u_{2}',u_{2}u_{3}\}

In [64]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(6)) + "\n" + r"\end{itemize}"))

\begin{itemize}
	\item $\mathcal{T}_{0} = \{1\}$
	\item $\mathcal{T}_{1} = \{\}$
	\item $\mathcal{T}_{2} = \{u_{2}\}$
	\item $\mathcal{T}_{3} = \{u_{2}',u_{3}\}$
	\item $\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}$
	\item $\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}u_{2}',u_{2}u_{3}\}$
\end{itemize}


In [65]:
## We create now the constants
C = C = [[],[]] + [[f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,6)]
RR = R.add_constants(*sum((c for c in C), []))
RwC = RR.base()
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,6)]

In [67]:
y_subs = {f"y_{i}" : sum(c*RR(t) for (c,t) in zip(C[i], T[i])) for i in range(2,6)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

\begin{array}{rcl}
	y_2 & \rightarrow & c_{2,0}u_{2}\\
	y_3 & \rightarrow & c_{3,0}u_{2}' + c_{3,1}u_{3}\\
	y_4 & \rightarrow & c_{4,2}u_{2}^2 + c_{4,0}u_{2}'' + c_{4,1}u_{3}'\\
	y_5 & \rightarrow & c_{5,2}u_{2}u_{2}' + c_{5,3}u_{2}u_{3} + c_{5,0}u_{2}''' + c_{5,1}u_{3}''
\end{array}


In [68]:
## We compute the linear system for each $m$
lin_systems = [sum(([c.wrapped for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(6)]

In [69]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.

In [70]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.


In [71]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.

In [72]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.


In [73]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{3,1} + 3c_{4,1} & = -6\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	-c_{2,0} + 6c_{4,2} & = 0\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.

In [74]:
print(_)

\left\{\begin{array}{rl}
	3c_{3,1} + 3c_{4,1} & = -6\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	-c_{2,0} + 6c_{4,2} & = 0\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.


In [75]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.

In [76]:
print(_)

\left\{\begin{array}{rl}
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.


In [77]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [78]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

0 \qquad\longrightarrow\qquad []

1 \qquad\longrightarrow\qquad []

2 \qquad\longrightarrow\qquad [c_{2,0} - \frac{2}{3}]

3 \qquad\longrightarrow\qquad [c_{2,0} - 1, c_{3,0}, c_{3,1} - 1]

4 \qquad\longrightarrow\qquad [c_{2,0} - \frac{4}{3},
 c_{3,0} - \frac{2}{3},
 c_{3,1} - \frac{4}{3},
 c_{4,0} - \frac{2}{9},
 c_{4,1} - \frac{2}{3},
 c_{4,2} - \frac{2}{9}]

5 \qquad\longrightarrow\qquad [c_{2,0} - \frac{5}{3},
 c_{3,0} - \frac{5}{3},
 c_{3,1} - \frac{5}{3},
 c_{4,0} - \frac{10}{9},
 c_{4,1} - \frac{5}{3},
 c_{4,2} - \frac{5}{9},
 c_{5,0},
 c_{5,1} - \frac{10}{9},
 c_{5,2},
 c_{5,3} - \frac{10}{9}]

#### **3. Computing the hierarchy equations**

In [79]:
R

Ring of operator polynomials in (u_2, u_3, y_2, y_3, y_4, y_5, z) over Differential Ring [[Rational Field], (0,)]

In [80]:
y_subs_true = [dict(),dict()] + [{f"y_{i}" : sum(R.base()(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,6)]

In [82]:
y_subs_true

[{},
 {},
 {'y_2': 2/3*u_2_0},
 {'y_2': u_2_0, 'y_3': u_3_0},
 {'y_2': 4/3*u_2_0,
  'y_3': 2/3*u_2_1 + 4/3*u_3_0,
  'y_4': 2/9*u_2_0^2 + 2/9*u_2_2 + 2/3*u_3_1},
 {'y_2': 5/3*u_2_0,
  'y_3': 5/3*u_2_1 + 5/3*u_3_0,
  'y_4': 5/9*u_2_0^2 + 10/9*u_2_2 + 5/3*u_3_1,
  'y_5': 10/9*u_2_0*u_3_0 + 10/9*u_3_2}]

In [83]:
H = [[lie_brackets[m].coefficient_full(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(6)]

##### m = 0

In [84]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.

In [85]:
print(_)

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 1

In [86]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.

In [87]:
print(_)

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.


##### m = 2

In [88]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}' + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.

In [89]:
print(_)

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}' + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.


##### m = 3

In [90]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.

In [91]:
print(_)

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 4

In [92]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}''' - \frac{2}{3}u_{2}u_{3}'' + \frac{4}{9}u_{2}^2u_{2}' + \frac{4}{3}u_{2}'u_{2}'' - \frac{2}{3}u_{2}'u_{3}' + \frac{2}{9}u_{2}^{(5)} - \frac{4}{3}u_{3}u_{3}' - \frac{1}{3}u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}'' - \frac{4}{3}u_{2}u_{3}' - \frac{4}{3}u_{2}'u_{3} + \frac{2}{3}u_{2}'^2 + \frac{1}{3}u_{2}^{(4)} - \frac{2}{3}u_{3}'''
\end{array}\right.

In [93]:
print(_)

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}''' - \frac{2}{3}u_{2}u_{3}'' + \frac{4}{9}u_{2}^2u_{2}' + \frac{4}{3}u_{2}'u_{2}'' - \frac{2}{3}u_{2}'u_{3}' + \frac{2}{9}u_{2}^{(5)} - \frac{4}{3}u_{3}u_{3}' - \frac{1}{3}u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}u_{2}'' - \frac{4}{3}u_{2}u_{3}' - \frac{4}{3}u_{2}'u_{3} + \frac{2}{3}u_{2}'^2 + \frac{1}{3}u_{2}^{(4)} - \frac{2}{3}u_{3}'''
\end{array}\right.


##### m = 5

In [94]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}u_{2}'u_{3} + \frac{5}{9}u_{2}u_{3}''' + \frac{5}{9}u_{2}^2u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{20}{9}u_{2}''u_{3}' + \frac{10}{9}u_{2}'''u_{3} - \frac{5}{3}u_{3}u_{3}'' - \frac{5}{3}u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}u_{2}''' + \frac{5}{9}u_{2}^2u_{2}' + \frac{5}{9}u_{2}'u_{2}'' + \frac{5}{3}u_{2}'u_{3}' + \frac{5}{3}u_{2}''u_{3} + \frac{1}{9}u_{2}^{(5)} - \frac{10}{3}u_{3}u_{3}'
\end{array}\right.

In [95]:
print(_)

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}u_{2}'u_{3} + \frac{5}{9}u_{2}u_{3}''' + \frac{5}{9}u_{2}^2u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{20}{9}u_{2}''u_{3}' + \frac{10}{9}u_{2}'''u_{3} - \frac{5}{3}u_{3}u_{3}'' - \frac{5}{3}u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}u_{2}''' + \frac{5}{9}u_{2}^2u_{2}' + \frac{5}{9}u_{2}'u_{2}'' + \frac{5}{3}u_{2}'u_{3}' + \frac{5}{3}u_{2}''u_{3} + \frac{1}{9}u_{2}^{(5)} - \frac{10}{3}u_{3}u_{3}'
\end{array}\right.


#### **Comparing with literature**

In [93]:
# q_1 -> u_2; q_0 -> u_3
subs_to_paper = {"u3": (1/2*u[2][1] + u[3])}
str_to_paper = lambda el : LatexExpr(poly_to_latex(el).replace("u_{2}", "q_{1}").replace("u_{3}", "q_{0}"))

In [94]:
# [P_r, L_3] = 3 f_n+1' \partial + 3/2 f_n+1'' + 3g_n+1'
# r = 3n+1 or 3n+2
# So in our case, for r=3n+l
# 3 f_{n+1}' = -H_{r, 1}; 3/2 f_{n+1}'' + 3g_{n+1}' = -H_{r, 0}
# Solving this we get
# f_{n+1} = -\int (1/3) H_{r,1}
# g_{n+1} = -\int (1/3)H_{r, 0} - 1/2 f_{n+1}'

In [100]:
# r = 1 (mod 3)
# f_1 = \int (1/3) H_{1,1}
# g_1 = \int (1/3) H_{1,0} - 1/2 f_1
f_1 = -((1/3)*H[1][1](**subs_to_paper)).inverse_operation(0)
g_1 = -((1/3)*H[1][0](**subs_to_paper)).inverse_operation(0) - (1/2)*f_1

In [101]:
str_to_paper(f_1)

\frac{1}{3}q_{1}

In [102]:
str_to_paper(H[1][0])

-q_{0}'

In [103]:
str_to_paper(g_1)

\frac{1}{6}q_{1}' - \frac{1}{6}q_{1} + \frac{1}{3}q_{0}

In [ ]:
# r = 1 (mod 3)
# f_2 = \int (1/3) H_{4,1}
# g_2 = \int (1/3) H_{4,0} - 1/2 f_2

In [ ]:
# r = 2 (mod 3)
# f_1 = \int (1/3) H_{2,1}
# g_1 = \int (1/3) H_{2,0} - 1/2 f_1

In [ ]:
# r = 2 (mod 3)
# f_2 = \int (1/3) H_{5,1}
# g_2 = \int (1/3) H_{5,0} - 1/2 f_2

## Case of order 5

In [96]:
## Setting up the framework to work
R.<u_2,u_3,u_4,u_5,y_2,y_3,y_4,y_5,y_6,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u_2, u_3, u_4, u_5)
y = (1, 0, y_2, y_3, y_4, y_5, y_6)
L = sum(z[i]*u[5-i] for i in range(6))
## Showing L
LatexExpr(poly_to_latex(L))

u_{2}\partial^{3} + u_{3}\partial^{2} + u_{4}\partial^{1} + u_{5} + \partial^{5}

In [97]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 6): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

\tilde{P}_{0}(U,Y) \longrightarrow 1

\tilde{P}_{1}(U,Y) \longrightarrow \partial^{1}

\tilde{P}_{2}(U,Y) \longrightarrow y_{2} + \partial^{2}

\tilde{P}_{3}(U,Y) \longrightarrow y_{2}\partial^{1} + y_{3} + \partial^{3}

\tilde{P}_{4}(U,Y) \longrightarrow y_{2}\partial^{2} + y_{3}\partial^{1} + y_{4} + \partial^{4}

\tilde{P}_{5}(U,Y) \longrightarrow y_{2}\partial^{3} + y_{3}\partial^{2} + y_{4}\partial^{1} + y_{5} + \partial^{5}

\tilde{P}_{6}(U,Y) \longrightarrow y_{2}\partial^{4} + y_{3}\partial^{3} + y_{4}\partial^{2} + y_{5}\partial^{1} + y_{6} + \partial^{6}

#### **1. Computing the differential systems**

In [98]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [99]:
diff_systems = [tuple([bracket.coefficient_full(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [100]:
bracket = lie_brackets[1]
LatexExpr(
    r"[L_5, \tilde{P}_1(U)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^3+\left(-u_{3}'\right)\partial^2+\left(-u_{4}'\right)\partial^1+\left(-u_{5}'\right)

In [101]:
print(_)

[L_5, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^3+\left(-u_{3}'\right)\partial^2+\left(-u_{4}'\right)\partial^1+\left(-u_{5}'\right)


In [102]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

()

##### The system for $m=2$

In [104]:
bracket = lie_brackets[2]
LatexExpr(
    r"[L_5, \tilde{P}_2(U, y_2)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 5y_{2}'\right)\partial^4+\left(-u_{2}'' - 2u_{3}' + 10y_{2}''\right)\partial^3+\left(3u_{2}y_{2}' - u_{3}'' - 2u_{4}' + 10y_{2}'''\right)\partial^2+\left(3u_{2}y_{2}'' + 2u_{3}y_{2}' - u_{4}'' - 2u_{5}' + 5y_{2}^{(4)}\right)\partial^1+\left(u_{2}y_{2}''' + u_{3}y_{2}'' + u_{4}y_{2}' - u_{5}'' + y_{2}^{(5)}\right)

In [105]:
print(_)

[L_5, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 5y_{2}'\right)\partial^4+\left(-u_{2}'' - 2u_{3}' + 10y_{2}''\right)\partial^3+\left(3u_{2}y_{2}' - u_{3}'' - 2u_{4}' + 10y_{2}'''\right)\partial^2+\left(3u_{2}y_{2}'' + 2u_{3}y_{2}' - u_{4}'' - 2u_{5}' + 5y_{2}^{(4)}\right)\partial^1+\left(u_{2}y_{2}''' + u_{3}y_{2}'' + u_{4}y_{2}' - u_{5}'' + y_{2}^{(5)}\right)


In [106]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[2]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [107]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 3$

In [108]:
bracket = lie_brackets[3]
LatexExpr(
    r"[L_5, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 5y_{2}'\right)\partial^5+\left(-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^4+\left(3u_{2}y_{2}' - u_{2}'y_{2} - u_{2}''' - 3u_{3}'' - 3u_{4}' + 10y_{2}''' + 10y_{3}''\right)\partial^3+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' + 2u_{3}y_{2}' - u_{3}'y_{2} - u_{3}''' - 3u_{4}'' - 3u_{5}' + 5y_{2}^{(4)} + 10y_{3}'''\right)\partial^2+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + u_{3}y_{2}'' + 2u_{3}y_{3}' + u_{4}y_{2}' - u_{4}'y_{2} - u_{4}''' - 3u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)}\right)\partial^1+\left(u_{2}y_{3}''' + u_{3}y_{3}'' + u_{4}y_{3}' - u_{5}'y_{2} - u_{5}''' + y_{3}^{(5)}\right)

In [109]:
print(_)

[L_5, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 5y_{2}'\right)\partial^5+\left(-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^4+\left(3u_{2}y_{2}' - u_{2}'y_{2} - u_{2}''' - 3u_{3}'' - 3u_{4}' + 10y_{2}''' + 10y_{3}''\right)\partial^3+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' + 2u_{3}y_{2}' - u_{3}'y_{2} - u_{3}''' - 3u_{4}'' - 3u_{5}' + 5y_{2}^{(4)} + 10y_{3}'''\right)\partial^2+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + u_{3}y_{2}'' + 2u_{3}y_{3}' + u_{4}y_{2}' - u_{4}'y_{2} - u_{4}''' - 3u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)}\right)\partial^1+\left(u_{2}y_{3}''' + u_{3}y_{3}'' + u_{4}y_{3}' - u_{5}'y_{2} - u_{5}''' + y_{3}^{(5)}\right)


In [110]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[3]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-3u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [111]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-3u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 4$

In [112]:
bracket = lie_brackets[4]
LatexExpr(
    r"[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 5y_{2}'\right)\partial^6+\left(-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^5+\left(3u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^4+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - u_{2}'y_{3} - u_{2}''y_{2} - u_{2}^{(4)} + 2u_{3}y_{2}' - 2u_{3}'y_{2} - 4u_{3}''' - 6u_{4}'' - 4u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}''\right)\partial^3+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' + u_{3}y_{2}'' + 2u_{3}y_{3}' - u_{3}'y_{3} - u_{3}''y_{2} - u_{3}^{(4)} + u_{4}y_{2}' - 2u_{4}'y_{2} - 4u_{4}''' - 6u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}'''\right)\partial^2+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + u_{3}y_{3}'' + 2u_{3}y_{4}' + u_{4}y_{3}' - u_{4}'y_{3} - u_{4}''y_{2} - u_{4}^{(4)} - 2u_{5}'y_{2} - 4u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)}\right)\partial^1+\left(u_{2}y_{4}''' + u_{3}y_{4}'' + u_{4}y_{4}' - u_{5}'y_{3} - u_{5}''y_{2} - u_{5}^{(4)} + y_{4}^{(5)}\right)

In [113]:
print(_)

[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 5y_{2}'\right)\partial^6+\left(-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^5+\left(3u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^4+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - u_{2}'y_{3} - u_{2}''y_{2} - u_{2}^{(4)} + 2u_{3}y_{2}' - 2u_{3}'y_{2} - 4u_{3}''' - 6u_{4}'' - 4u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}''\right)\partial^3+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' + u_{3}y_{2}'' + 2u_{3}y_{3}' - u_{3}'y_{3} - u_{3}''y_{2} - u_{3}^{(4)} + u_{4}y_{2}' - 2u_{4}'y_{2} - 4u_{4}''' - 6u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}'''\right)\partial^2+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + u_{3}y_{3}'' + 2u_{3}y_{4}' + u_{4}y_{3}' - u_{4}'y_{3} - u_{4}''y_{2} - u_{4}^{(4)} - 2u_{5}'y_{2} - 4u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)}\right)\partial^1+\left(u_{2}y_{4}''' + u_{3}y_{4}'' + u_{4}y_{4}' - u_{5}'y_{3} - u_{5}''y_{2} - u_{5}^{(4)} + y_{4}^{(5)}\ri

In [114]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[4]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	3u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-4u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [115]:
print(_)

\left\{\begin{array}{rl}
	3u_{2}y_{2}' - 2u_{2}'y_{2} - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-4u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 5$

In [117]:
bracket = lie_brackets[5]
LatexExpr(
    r"[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") + "\n\t" 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 5y_{2}'\right)\partial^7
	+\left(-10u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^6
	+\left(3u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^5
	+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} + 2u_{3}y_{2}' - 3u_{3}'y_{2} - 10u_{3}''' - 10u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^4
	+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - u_{2}'y_{4} - u_{2}''y_{3} - u_{2}'''y_{2} - u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 2u_{3}'y_{3} - 3u_{3}''y_{2} - 5u_{3}^{(4)} + u_{4}y_{2}' - 3u_{4}'y_{2} - 10u_{4}''' - 10u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}''\right)\partial^3
	+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' + u_{3}y_{3}'' + 2u_{3}y_{4}' - u_{3}'y_{4} - u_{3}''y_{3} - u_{3}'''y_{2} - u_{3}^{(5)} + u_{4}y_{3}' - 2u_{4}'y_{3} - 3u_{4}''y_{2} - 5u_{4}^{(4)} - 3u_{5}'y_{2} - 10u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)} + 10y_{5}'''\right)\partial^2
	+\left(u_{2}y_{4}''' + 3u_{2}y_{5}'' + u_{3}y_{4}'' + 2u_{3}y_{5}' + u_{4}y_{4}' - u_{4}'y_{4} - u_{4}''y_{3} - u_{4}'''y_{2} - u_{4}^{(5)} - 2u_{5}'y_{3} - 3u_{5}''y_{2} - 5u_{5}^{(4)} + y_{4}^{(5)} + 5y_{5}^{(4)}\right)\partial^1
	+\left(u_{2}y_{5}''' + u_{3}y_{5}'' + u_{4}y_{5}' - u_{5}'y_{4} - u_{5}''y_{3} - u_{5}'''y_{2} - u_{5}^{(5)} + y_{5}^{(5)}\right)

In [118]:
print(_)

[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 5y_{2}'\right)\partial^7
	+\left(-10u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^6
	+\left(3u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^5
	+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} + 2u_{3}y_{2}' - 3u_{3}'y_{2} - 10u_{3}''' - 10u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^4
	+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - u_{2}'y_{4} - u_{2}''y_{3} - u_{2}'''y_{2} - u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 2u_{3}'y_{3} - 3u_{3}''y_{2} - 5u_{3}^{(4)} + u_{4}y_{2}' - 3u_{4}'y_{2} - 10u_{4}''' - 10u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}''\right)\partial^3
	+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' + u_{3}y_{3}'' + 2u_{3}y_{4}' - u_{3}'y_{4} - u_{3}''y_{3} - u_{3}'''y_{2} - u_{3}^{(5)} + u_{4}y_{3}' - 2u_{4}'y_{3} - 3u_{4}''y_{2} - 5u_{4}^{(

In [119]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[5]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} + 2u_{3}y_{2}' - 3u_{3}'y_{2} - 10u_{3}''' - 10u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	3u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-5u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [120]:
print(_)

\left\{\begin{array}{rl}
	3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{2}'y_{3} - 3u_{2}''y_{2} - 5u_{2}^{(4)} + 2u_{3}y_{2}' - 3u_{3}'y_{2} - 10u_{3}''' - 10u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	3u_{2}y_{2}' - 3u_{2}'y_{2} - 10u_{2}''' - 10u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-10u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-5u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 6$

In [122]:
bracket = lie_brackets[6]
LatexExpr(
    r"[L_5, \tilde{P}_6(U, Y)] = " + "+".join(
        r"\left(" + poly_to_latex(bracket.coefficient_full(z[i])) + r"\right)" + 
        (r"\partial^" + str(i) if i > 0 else "") + "\n\t" 
        for i in range(bracket.order(z), -1, -1) if bracket.coefficient_full(z[i]) != 0
    )
)

[L_5, \tilde{P}_6(U, Y)] = \left(-6u_{2}' + 5y_{2}'\right)\partial^8
	+\left(-15u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^7
	+\left(3u_{2}y_{2}' - 4u_{2}'y_{2} - 20u_{2}''' - 15u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^6
	+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{2}'y_{3} - 6u_{2}''y_{2} - 15u_{2}^{(4)} + 2u_{3}y_{2}' - 4u_{3}'y_{2} - 20u_{3}''' - 15u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^5
	+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 2u_{2}'y_{4} - 3u_{2}''y_{3} - 4u_{2}'''y_{2} - 6u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{3}'y_{3} - 6u_{3}''y_{2} - 15u_{3}^{(4)} + u_{4}y_{2}' - 4u_{4}'y_{2} - 20u_{4}''' - 15u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'\right)\partial^4
	+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - u_{2}'y_{5} - u_{2}''y_{4} - u_{2}'''y_{3} - u_{2}^{(4)}y_{2} - u_{2}^{(6)} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 2u_{3}'y_{4} - 3u_{3}''y_{3} - 4u_{3}'''y_{2} - 6u_{3}^{(5)} + u_{4}y_{3}' - 3u_{4}'y_{3} - 6u_{4}''y_{2} - 15u_{4}^{(4)} - 4u_{5}'y_{2} - 20u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)} + 10y_{5}''' + 10y_{6}''\right)\partial^3
	+\left(u_{2}y_{4}''' + 3u_{2}y_{5}'' + 3u_{2}y_{6}' + u_{3}y_{4}'' + 2u_{3}y_{5}' - u_{3}'y_{5} - u_{3}''y_{4} - u_{3}'''y_{3} - u_{3}^{(4)}y_{2} - u_{3}^{(6)} + u_{4}y_{4}' - 2u_{4}'y_{4} - 3u_{4}''y_{3} - 4u_{4}'''y_{2} - 6u_{4}^{(5)} - 3u_{5}'y_{3} - 6u_{5}''y_{2} - 15u_{5}^{(4)} + y_{4}^{(5)} + 5y_{5}^{(4)} + 10y_{6}'''\right)\partial^2
	+\left(u_{2}y_{5}''' + 3u_{2}y_{6}'' + u_{3}y_{5}'' + 2u_{3}y_{6}' + u_{4}y_{5}' - u_{4}'y_{5} - u_{4}''y_{4} - u_{4}'''y_{3} - u_{4}^{(4)}y_{2} - u_{4}^{(6)} - 2u_{5}'y_{4} - 3u_{5}''y_{3} - 4u_{5}'''y_{2} - 6u_{5}^{(5)} + y_{5}^{(5)} + 5y_{6}^{(4)}\right)\partial^1
	+\left(u_{2}y_{6}''' + u_{3}y_{6}'' + u_{4}y_{6}' - u_{5}'y_{5} - u_{5}''y_{4} - u_{5}'''y_{3} - u_{5}^{(4)}y_{2} - u_{5}^{(6)} + y_{6}^{(5)}\right)

In [123]:
print(_)

[L_5, \tilde{P}_6(U, Y)] = \left(-6u_{2}' + 5y_{2}'\right)\partial^8
	+\left(-15u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^7
	+\left(3u_{2}y_{2}' - 4u_{2}'y_{2} - 20u_{2}''' - 15u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^6
	+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{2}'y_{3} - 6u_{2}''y_{2} - 15u_{2}^{(4)} + 2u_{3}y_{2}' - 4u_{3}'y_{2} - 20u_{3}''' - 15u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^5
	+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 2u_{2}'y_{4} - 3u_{2}''y_{3} - 4u_{2}'''y_{2} - 6u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{3}'y_{3} - 6u_{3}''y_{2} - 15u_{3}^{(4)} + u_{4}y_{2}' - 4u_{4}'y_{2} - 20u_{4}''' - 15u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'\right)\partial^4
	+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - u_{2}'y_{5} - u_{2}''y_{4} - u_{2}'''y_{3} - u_{2}^{(4)}y_{2} - u_{2}^{(6)} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 2u_{3}'y_{4} - 3u_{3}''y_{3} - 4u

In [124]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[6]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 2u_{2}'y_{4} - 3u_{2}''y_{3} - 4u_{2}'''y_{2} - 6u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{3}'y_{3} - 6u_{3}''y_{2} - 15u_{3}^{(4)} + u_{4}y_{2}' - 4u_{4}'y_{2} - 20u_{4}''' - 15u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'&= 0\\
	3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{2}'y_{3} - 6u_{2}''y_{2} - 15u_{2}^{(4)} + 2u_{3}y_{2}' - 4u_{3}'y_{2} - 20u_{3}''' - 15u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	3u_{2}y_{2}' - 4u_{2}'y_{2} - 20u_{2}''' - 15u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-15u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-6u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [125]:
print(_)

\left\{\begin{array}{rl}
	u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 2u_{2}'y_{4} - 3u_{2}''y_{3} - 4u_{2}'''y_{2} - 6u_{2}^{(5)} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{3}'y_{3} - 6u_{3}''y_{2} - 15u_{3}^{(4)} + u_{4}y_{2}' - 4u_{4}'y_{2} - 20u_{4}''' - 15u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'&= 0\\
	3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{2}'y_{3} - 6u_{2}''y_{2} - 15u_{2}^{(4)} + 2u_{3}y_{2}' - 4u_{3}'y_{2} - 20u_{3}''' - 15u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	3u_{2}y_{2}' - 4u_{2}'y_{2} - 20u_{2}''' - 15u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-15u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-6u_{2}' + 5y_{2}'&= 0
\end{array}\right.


#### **2. Computing the basis of $W(L_5)$**

In [126]:
W = R.weight_func([2,3,4,5,0,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(7)]

In [127]:
for i in range(7):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

\mathcal{T}_{0} = \{1\}

\mathcal{T}_{1} = \{\}

\mathcal{T}_{2} = \{u_{2}\}

\mathcal{T}_{3} = \{u_{2}',u_{3}\}

\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2,u_{4}\}

\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}u_{2}',u_{4}',u_{2}u_{3},u_{5}\}

\mathcal{T}_{6} = \{u_{2}^{(4)},u_{3}''',u_{2}'^2,u_{2}u_{2}'',u_{4}'',u_{2}'u_{3},u_{2}u_{3}',u_{5}',u_{2}^3,u_{2}u_{4},u_{3}^2\}

In [128]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(7)) + "\n" + r"\end{itemize}"))

\begin{itemize}
	\item $\mathcal{T}_{0} = \{1\}$
	\item $\mathcal{T}_{1} = \{\}$
	\item $\mathcal{T}_{2} = \{u_{2}\}$
	\item $\mathcal{T}_{3} = \{u_{2}',u_{3}\}$
	\item $\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2,u_{4}\}$
	\item $\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}u_{2}',u_{4}',u_{2}u_{3},u_{5}\}$
	\item $\mathcal{T}_{6} = \{u_{2}^{(4)},u_{3}''',u_{2}'^2,u_{2}u_{2}'',u_{4}'',u_{2}'u_{3},u_{2}u_{3}',u_{5}',u_{2}^3,u_{2}u_{4},u_{3}^2\}$
\end{itemize}


In [130]:
## We create now the constants
C = C = [[],[]] + [[f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,7)]
RR = R.add_constants(*sum((c for c in C), []))
RwC = RR.base()
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,7)]

In [131]:
y_subs = {f"y_{i}" : sum(c*RR(t) for (c,t) in zip(C[i], T[i])) for i in range(2,7)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

\begin{array}{rcl}
	y_2 & \rightarrow & c_{2,0}u_{2}\\
	y_3 & \rightarrow & c_{3,0}u_{2}' + c_{3,1}u_{3}\\
	y_4 & \rightarrow & c_{4,2}u_{2}^2 + c_{4,0}u_{2}'' + c_{4,1}u_{3}' + c_{4,3}u_{4}\\
	y_5 & \rightarrow & c_{5,2}u_{2}u_{2}' + c_{5,4}u_{2}u_{3} + c_{5,0}u_{2}''' + c_{5,1}u_{3}'' + c_{5,3}u_{4}' + c_{5,5}u_{5}\\
	y_6 & \rightarrow & c_{6,3}u_{2}u_{2}'' + c_{6,6}u_{2}u_{3}' + c_{6,9}u_{2}u_{4} + c_{6,8}u_{2}^3 + c_{6,5}u_{2}'u_{3} + c_{6,2}u_{2}'^2 + c_{6,0}u_{2}^{(4)} + c_{6,10}u_{3}^2 + c_{6,1}u_{3}''' + c_{6,4}u_{4}'' + c_{6,7}u_{5}'
\end{array}


In [132]:
## We compute the linear system for each $m$
lin_systems = [sum(([c.wrapped for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(7)]

In [133]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	5c_{2,0} & = 2
\end{array}\right.

In [134]:
print(_)

\left\{\begin{array}{rl}
	5c_{2,0} & = 2
\end{array}\right.


In [135]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	10c_{2,0} + 5c_{3,0} & = 3\\
	5c_{3,1} & = 3\\
	5c_{2,0} & = 3
\end{array}\right.

In [136]:
print(_)

\left\{\begin{array}{rl}
	10c_{2,0} + 5c_{3,0} & = 3\\
	5c_{3,1} & = 3\\
	5c_{2,0} & = 3
\end{array}\right.


In [137]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = 4\\
	10c_{3,1} + 5c_{4,1} & = 6\\
	c_{2,0} + 10c_{4,2} & = 0\\
	5c_{4,3} & = 4\\
	10c_{2,0} + 5c_{3,0} & = 6\\
	5c_{3,1} & = 4\\
	5c_{2,0} & = 4
\end{array}\right.

In [138]:
print(_)

\left\{\begin{array}{rl}
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = 4\\
	10c_{3,1} + 5c_{4,1} & = 6\\
	c_{2,0} + 10c_{4,2} & = 0\\
	5c_{4,3} & = 4\\
	10c_{2,0} + 5c_{3,0} & = 6\\
	5c_{3,1} & = 4\\
	5c_{2,0} & = 4
\end{array}\right.


In [139]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	2c_{2,0} - 2c_{3,1} + 5c_{5,4} & = 0\\
	-3c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	10c_{4,3} + 5c_{5,3} & = 10\\
	3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-2c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = 10\\
	5c_{5,5} & = 5\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = 5\\
	10c_{3,1} + 5c_{4,1} & = 10\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = 10\\
	10c_{4,2} & = 0\\
	5c_{4,3} & = 5\\
	10c_{2,0} + 5c_{3,0} & = 10\\
	5c_{3,1} & = 5\\
	5c_{2,0} & = 5
\end{array}\right.

In [140]:
print(_)

\left\{\begin{array}{rl}
	2c_{2,0} - 2c_{3,1} + 5c_{5,4} & = 0\\
	-3c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	10c_{4,3} + 5c_{5,3} & = 10\\
	3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-2c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = 10\\
	5c_{5,5} & = 5\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = 5\\
	10c_{3,1} + 5c_{4,1} & = 10\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = 10\\
	10c_{4,2} & = 0\\
	5c_{4,3} & = 5\\
	10c_{2,0} + 5c_{3,0} & = 10\\
	5c_{3,1} & = 5\\
	5c_{2,0} & = 5
\end{array}\right.


In [141]:
len(lin_systems[6])

28

In [142]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(-equ.constant_coefficient()) 
        for equ in lin_systems[6]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	c_{2,0} + 2c_{3,0} - 3c_{3,1} + 10c_{5,4} + 5c_{6,5} & = 0\\
	-4c_{2,0} + 3c_{4,3} + 5c_{6,9} & = 0\\
	c_{2,0} + 5c_{3,0} + 10c_{4,0} + 10c_{5,0} + 5c_{6,0} & = 6\\
	-3c_{3,0} - 2c_{4,1} + 20c_{5,4} + 5c_{6,5} + 5c_{6,6} & = 0\\
	5c_{3,1} + 10c_{4,1} + 10c_{5,1} + 5c_{6,1} & = 15\\
	-c_{3,1} + 10c_{6,10} & = 0\\
	-6c_{2,0} + 3c_{3,1} + 3c_{4,1} + 10c_{5,4} + 5c_{6,6} & = 0\\
	-3c_{2,0} + 3c_{3,0} + 3c_{4,0} + 20c_{4,2} + 10c_{5,2} + 5c_{6,3} & = 0\\
	c_{2,0} - 2c_{4,3} + 5c_{6,9} & = 0\\
	10c_{5,5} + 5c_{6,7} & = 15\\
	4c_{4,2} + 15c_{6,8} & = 0\\
	-3c_{3,0} - 2c_{4,0} + 60c_{4,2} + 30c_{5,2} + 10c_{6,2} + 5c_{6,3} & = 0\\
	10c_{4,3} + 10c_{5,3} + 5c_{6,4} & = 20\\
	2c_{2,0} - 3c_{3,1} + 5c_{5,4} & = 0\\
	-4c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	10c_{4,3} + 5c_{5,3} & = 15\\
	-3c_{2,0} + 3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = 20\\
	5c_{5,5} & = 6\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = 15\\
	10c_{3,1} + 5c_{4,1} & = 15\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = 20\\
	-c_{2,0} + 10c_{4,2} & = 0\\
	5c_{4,3} & = 6\\
	10c_{2,0} + 5c_{3,0} & = 15\\
	5c_{3,1} & = 6\\
	5c_{2,0} & = 6
\end{array}\right.

In [143]:
print(_)

\left\{\begin{array}{rl}
	c_{2,0} + 2c_{3,0} - 3c_{3,1} + 10c_{5,4} + 5c_{6,5} & = 0\\
	-4c_{2,0} + 3c_{4,3} + 5c_{6,9} & = 0\\
	c_{2,0} + 5c_{3,0} + 10c_{4,0} + 10c_{5,0} + 5c_{6,0} & = 6\\
	-3c_{3,0} - 2c_{4,1} + 20c_{5,4} + 5c_{6,5} + 5c_{6,6} & = 0\\
	5c_{3,1} + 10c_{4,1} + 10c_{5,1} + 5c_{6,1} & = 15\\
	-c_{3,1} + 10c_{6,10} & = 0\\
	-6c_{2,0} + 3c_{3,1} + 3c_{4,1} + 10c_{5,4} + 5c_{6,6} & = 0\\
	-3c_{2,0} + 3c_{3,0} + 3c_{4,0} + 20c_{4,2} + 10c_{5,2} + 5c_{6,3} & = 0\\
	c_{2,0} - 2c_{4,3} + 5c_{6,9} & = 0\\
	10c_{5,5} + 5c_{6,7} & = 15\\
	4c_{4,2} + 15c_{6,8} & = 0\\
	-3c_{3,0} - 2c_{4,0} + 60c_{4,2} + 30c_{5,2} + 10c_{6,2} + 5c_{6,3} & = 0\\
	10c_{4,3} + 10c_{5,3} + 5c_{6,4} & = 20\\
	2c_{2,0} - 3c_{3,1} + 5c_{5,4} & = 0\\
	-4c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	10c_{4,3} + 5c_{5,3} & = 15\\
	-3c_{2,0} + 3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = 20\\
	5c_{5,5} & = 6\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0

In [144]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [145]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

0 \qquad\longrightarrow\qquad []

1 \qquad\longrightarrow\qquad []

2 \qquad\longrightarrow\qquad [c_{2,0} - \frac{2}{5}]

3 \qquad\longrightarrow\qquad [c_{2,0} - \frac{3}{5}, c_{3,0} + \frac{3}{5}, c_{3,1} - \frac{3}{5}]

4 \qquad\longrightarrow\qquad [c_{2,0} - \frac{4}{5},
 c_{3,0} + \frac{2}{5},
 c_{3,1} - \frac{4}{5},
 c_{4,0},
 c_{4,1} + \frac{2}{5},
 c_{4,2} + \frac{2}{25},
 c_{4,3} - \frac{4}{5}]

5 \qquad\longrightarrow\qquad [c_{2,0} - 1,
 c_{3,0},
 c_{3,1} - 1,
 c_{4,0},
 c_{4,1},
 c_{4,2},
 c_{4,3} - 1,
 c_{5,0},
 c_{5,1},
 c_{5,2},
 c_{5,3},
 c_{5,4},
 c_{5,5} - 1]

6 \qquad\longrightarrow\qquad [c_{2,0} - \frac{6}{5},
 c_{3,0} - \frac{3}{5},
 c_{3,1} - \frac{6}{5},
 c_{4,0} - \frac{2}{5},
 c_{4,1} - \frac{3}{5},
 c_{4,2} - \frac{3}{25},
 c_{4,3} - \frac{6}{5},
 c_{5,0} + \frac{1}{5},
 c_{5,1} - \frac{2}{5},
 c_{5,2} + \frac{3}{25},
 c_{5,3} - \frac{3}{5},
 c_{5,4} - \frac{6}{25},
 c_{5,5} - \frac{6}{5},
 c_{6,0} + \frac{1}{25},
 c_{6,1} + \frac{1}{5},
 c_{6,2} + \frac{1}{25},
 c_{6,3} + \frac{3}{25},
 c_{6,4} - \frac{2}{5},
 c_{6,5} + \frac{6}{25},
 c_{6,6} + \frac{3}{25},
 c_{6,7} - \frac{3}{5},
 c_{6,8} + \frac{4}{125},
 c_{6,9} - \frac{6}{25},
 c_{6,10} - \frac{3}{25}]

#### **3. Computing the hierarchy equations**

In [146]:
y_subs_true = [dict(),dict()] + [{f"y_{i}" : sum(R.base()(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,7)]

In [148]:
H = [[lie_brackets[m].coefficient_full(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(7)]

##### m = 0

In [149]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.

In [150]:
print(_)

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.


##### m = 1

In [151]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3,u_4,u_5) = & -u_{5}'\\
	H_{1,1}(u_2,u_3,u_4,u_5) = & -u_{4}'\\
	H_{1,2}(u_2,u_3,u_4,u_5) = & -u_{3}'\\
	H_{1,3}(u_2,u_3,u_4,u_5) = & -u_{2}'
\end{array}\right.

In [152]:
print(_)

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3,u_4,u_5) = & -u_{5}'\\
	H_{1,1}(u_2,u_3,u_4,u_5) = & -u_{4}'\\
	H_{1,2}(u_2,u_3,u_4,u_5) = & -u_{3}'\\
	H_{1,3}(u_2,u_3,u_4,u_5) = & -u_{2}'
\end{array}\right.


##### m = 2

In [153]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3,u_4,u_5) = & \frac{2}{5}u_{2}u_{2}''' + \frac{2}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}''u_{3} + \frac{2}{5}u_{2}^{(5)} - u_{5}''\\
	H_{2,1}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}'' + \frac{4}{5}u_{2}'u_{3} + 2u_{2}^{(4)} - u_{4}'' - 2u_{5}'\\
	H_{2,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}' + 4u_{2}''' - u_{3}'' - 2u_{4}'\\
	H_{2,3}(u_2,u_3,u_4,u_5) = & 3u_{2}'' - 2u_{3}'
\end{array}\right.

In [154]:
print(_)

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3,u_4,u_5) = & \frac{2}{5}u_{2}u_{2}''' + \frac{2}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}''u_{3} + \frac{2}{5}u_{2}^{(5)} - u_{5}''\\
	H_{2,1}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}'' + \frac{4}{5}u_{2}'u_{3} + 2u_{2}^{(4)} - u_{4}'' - 2u_{5}'\\
	H_{2,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}' + 4u_{2}''' - u_{3}'' - 2u_{4}'\\
	H_{2,3}(u_2,u_3,u_4,u_5) = & 3u_{2}'' - 2u_{3}'
\end{array}\right.


##### m = 3

In [155]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3,u_4,u_5) = & -\frac{3}{5}u_{2}u_{2}^{(4)} + \frac{3}{5}u_{2}u_{3}''' - \frac{3}{5}u_{2}u_{5}' - \frac{3}{5}u_{2}''u_{4} - \frac{3}{5}u_{2}'''u_{3} - \frac{3}{5}u_{2}^{(6)} + \frac{3}{5}u_{3}u_{3}'' + \frac{3}{5}u_{3}'u_{4} + \frac{3}{5}u_{3}^{(5)} - u_{5}'''\\
	H_{3,1}(u_2,u_3,u_4,u_5) = & -\frac{6}{5}u_{2}u_{2}''' + \frac{9}{5}u_{2}u_{3}'' - \frac{3}{5}u_{2}u_{4}' + \frac{3}{5}u_{2}'u_{4} - \frac{3}{5}u_{2}''u_{3} - \frac{12}{5}u_{2}^{(5)} + \frac{6}{5}u_{3}u_{3}' + 3u_{3}^{(4)} - u_{4}''' - 3u_{5}''\\
	H_{3,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{3}' + \frac{6}{5}u_{2}'u_{3} - 3u_{2}^{(4)} + 5u_{3}''' - 3u_{4}'' - 3u_{5}'\\
	H_{3,3}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}' - u_{2}''' + 3u_{3}'' - 3u_{4}'
\end{array}\right.

In [156]:
print(_)

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3,u_4,u_5) = & -\frac{3}{5}u_{2}u_{2}^{(4)} + \frac{3}{5}u_{2}u_{3}''' - \frac{3}{5}u_{2}u_{5}' - \frac{3}{5}u_{2}''u_{4} - \frac{3}{5}u_{2}'''u_{3} - \frac{3}{5}u_{2}^{(6)} + \frac{3}{5}u_{3}u_{3}'' + \frac{3}{5}u_{3}'u_{4} + \frac{3}{5}u_{3}^{(5)} - u_{5}'''\\
	H_{3,1}(u_2,u_3,u_4,u_5) = & -\frac{6}{5}u_{2}u_{2}''' + \frac{9}{5}u_{2}u_{3}'' - \frac{3}{5}u_{2}u_{4}' + \frac{3}{5}u_{2}'u_{4} - \frac{3}{5}u_{2}''u_{3} - \frac{12}{5}u_{2}^{(5)} + \frac{6}{5}u_{3}u_{3}' + 3u_{3}^{(4)} - u_{4}''' - 3u_{5}''\\
	H_{3,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{3}' + \frac{6}{5}u_{2}'u_{3} - 3u_{2}^{(4)} + 5u_{3}''' - 3u_{4}'' - 3u_{5}'\\
	H_{3,3}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}u_{2}' - u_{2}''' + 3u_{3}'' - 3u_{4}'
\end{array}\right.


##### m = 4

In [157]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3,u_4,u_5) = & -\frac{12}{25}u_{2}u_{2}'u_{2}'' - \frac{4}{25}u_{2}u_{2}'u_{4} - \frac{4}{25}u_{2}u_{2}''u_{3} - \frac{4}{25}u_{2}u_{2}^{(5)} - \frac{2}{5}u_{2}u_{3}^{(4)} + \frac{4}{5}u_{2}u_{4}''' - \frac{4}{5}u_{2}u_{5}'' - \frac{4}{25}u_{2}^2u_{2}''' - \frac{4}{5}u_{2}'u_{2}^{(4)} + \frac{2}{5}u_{2}'u_{5}' - \frac{4}{25}u_{2}'^2u_{3} - \frac{8}{5}u_{2}''u_{2}''' - \frac{2}{5}u_{3}u_{3}''' + \frac{4}{5}u_{3}u_{4}'' - \frac{4}{5}u_{3}u_{5}' - \frac{2}{5}u_{3}''u_{4} - \frac{2}{5}u_{3}^{(6)} + \frac{4}{5}u_{4}u_{4}' + \frac{4}{5}u_{4}^{(5)} - u_{5}^{(4)}\\
	H_{4,1}(u_2,u_3,u_4,u_5) = & -\frac{8}{25}u_{2}u_{2}'u_{3} - \frac{12}{25}u_{2}u_{2}'^2 - \frac{6}{5}u_{2}u_{2}^{(4)} - \frac{2}{5}u_{2}u_{3}''' + \frac{8}{5}u_{2}u_{4}'' - \frac{8}{5}u_{2}u_{5}' - \frac{12}{25}u_{2}^2u_{2}'' - \frac{16}{5}u_{2}'u_{2}''' + \frac{2}{5}u_{2}'u_{4}' - \frac{2}{5}u_{2}''u_{4} - \frac{12}{5}u_{2}''^2 - \frac{2}{5}u_{2}'''u_{3} - \frac{2}{5}u_{2}^{(6)} + \frac{4}{5}u_{3}u_{4}' + \frac{4}{5}u_{3}'u_{4} - \frac{6}{5}u_{3}^{(5)} + 3u_{4}^{(4)} - 4u_{5}'''\\
	H_{4,2}(u_2,u_3,u_4,u_5) = & -2u_{2}u_{2}''' + \frac{2}{5}u_{2}u_{3}'' + \frac{4}{5}u_{2}u_{4}' - \frac{12}{25}u_{2}^2u_{2}' - \frac{24}{5}u_{2}'u_{2}'' + \frac{2}{5}u_{2}'u_{3}' + \frac{4}{5}u_{2}'u_{4} - \frac{6}{5}u_{2}^{(5)} + \frac{4}{5}u_{3}u_{3}' - u_{3}^{(4)} + 4u_{4}''' - 6u_{5}''\\
	H_{4,3}(u_2,u_3,u_4,u_5) = & -\frac{6}{5}u_{2}u_{2}'' + \frac{4}{5}u_{2}u_{3}' + \frac{4}{5}u_{2}'u_{3} - \frac{6}{5}u_{2}'^2 - u_{2}^{(4)} + 2u_{4}'' - 4u_{5}'
\end{array}\right.

In [158]:
print(_)

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3,u_4,u_5) = & -\frac{12}{25}u_{2}u_{2}'u_{2}'' - \frac{4}{25}u_{2}u_{2}'u_{4} - \frac{4}{25}u_{2}u_{2}''u_{3} - \frac{4}{25}u_{2}u_{2}^{(5)} - \frac{2}{5}u_{2}u_{3}^{(4)} + \frac{4}{5}u_{2}u_{4}''' - \frac{4}{5}u_{2}u_{5}'' - \frac{4}{25}u_{2}^2u_{2}''' - \frac{4}{5}u_{2}'u_{2}^{(4)} + \frac{2}{5}u_{2}'u_{5}' - \frac{4}{25}u_{2}'^2u_{3} - \frac{8}{5}u_{2}''u_{2}''' - \frac{2}{5}u_{3}u_{3}''' + \frac{4}{5}u_{3}u_{4}'' - \frac{4}{5}u_{3}u_{5}' - \frac{2}{5}u_{3}''u_{4} - \frac{2}{5}u_{3}^{(6)} + \frac{4}{5}u_{4}u_{4}' + \frac{4}{5}u_{4}^{(5)} - u_{5}^{(4)}\\
	H_{4,1}(u_2,u_3,u_4,u_5) = & -\frac{8}{25}u_{2}u_{2}'u_{3} - \frac{12}{25}u_{2}u_{2}'^2 - \frac{6}{5}u_{2}u_{2}^{(4)} - \frac{2}{5}u_{2}u_{3}''' + \frac{8}{5}u_{2}u_{4}'' - \frac{8}{5}u_{2}u_{5}' - \frac{12}{25}u_{2}^2u_{2}'' - \frac{16}{5}u_{2}'u_{2}''' + \frac{2}{5}u_{2}'u_{4}' - \frac{2}{5}u_{2}''u_{4} - \frac{12}{5}u_{2}''^2 - \frac{2}{5}u_{2}'''u_{3} - \frac{2}{5}u_{2}^{(6)} + \frac{4}{5

##### m = 5

In [159]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.

In [160]:
print(_)

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.


##### m = 6

In [161]:
m=6
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{6,0}(u_2,u_3,u_4,u_5) = & -\frac{7}{5}u_{2}u_{2}'u_{2}^{(4)} - \frac{3}{5}u_{2}u_{2}'u_{3}''' + \frac{18}{25}u_{2}u_{2}'u_{4}'' + \frac{3}{25}u_{2}u_{2}'u_{5}' - \frac{24}{125}u_{2}u_{2}'^2u_{3} - \frac{24}{125}u_{2}u_{2}'^3 - \frac{66}{25}u_{2}u_{2}''u_{2}''' - \frac{27}{25}u_{2}u_{2}''u_{3}'' + \frac{18}{25}u_{2}u_{2}''u_{4}' - \frac{21}{25}u_{2}u_{2}'''u_{3}' + \frac{3}{25}u_{2}u_{2}'''u_{4} - \frac{9}{25}u_{2}u_{2}^{(4)}u_{3} - \frac{4}{25}u_{2}u_{2}^{(7)} + \frac{3}{25}u_{2}u_{3}u_{3}''' + \frac{6}{25}u_{2}u_{3}u_{4}'' - \frac{6}{25}u_{2}u_{3}u_{5}' + \frac{18}{25}u_{2}u_{3}'u_{3}'' - \frac{3}{25}u_{2}u_{3}''u_{4} - \frac{8}{25}u_{2}u_{3}^{(6)} + \frac{6}{25}u_{2}u_{4}u_{4}' + \frac{16}{25}u_{2}u_{4}^{(5)} - \frac{3}{5}u_{2}u_{5}^{(4)} - \frac{72}{125}u_{2}^2u_{2}'u_{2}'' - \frac{12}{125}u_{2}^2u_{2}'u_{4} - \frac{12}{125}u_{2}^2u_{2}''u_{3} - \frac{27}{125}u_{2}^2u_{2}^{(5)} - \frac{3}{25}u_{2}^2u_{3}^{(4)} + \frac{6}{25}u_{2}^2u_{4}''' - \frac{3}{25}u_{2}^2u_{5}'' - \frac{12}{125}u_{2}^3u_{2}''' - \frac{1}{5}u_{2}'u_{2}''u_{4} - \frac{72}{25}u_{2}'u_{2}''^2 - \frac{8}{25}u_{2}'u_{2}'''u_{3} - \frac{17}{25}u_{2}'u_{2}^{(6)} - \frac{12}{25}u_{2}'u_{3}u_{3}'' + \frac{12}{25}u_{2}'u_{3}u_{4}' - \frac{9}{25}u_{2}'u_{3}'u_{4} - \frac{21}{25}u_{2}'u_{3}^{(5)} + \frac{6}{25}u_{2}'u_{4}^2 + \frac{6}{5}u_{2}'u_{4}^{(4)} - \frac{3}{5}u_{2}'u_{5}''' - \frac{48}{25}u_{2}'^2u_{2}''' - \frac{43}{25}u_{2}''u_{2}^{(5)} - \frac{3}{5}u_{2}''u_{3}u_{3}' - \frac{12}{5}u_{2}''u_{3}^{(4)} + \frac{12}{5}u_{2}''u_{4}''' - \frac{2}{5}u_{2}''u_{5}'' - \frac{1}{5}u_{2}''^2u_{3} - \frac{13}{5}u_{2}'''u_{2}^{(4)} - \frac{6}{25}u_{2}'''u_{3}^2 - \frac{18}{5}u_{2}'''u_{3}''' + \frac{12}{5}u_{2}'''u_{4}'' + \frac{1}{5}u_{2}'''u_{5}' - 3u_{2}^{(4)}u_{3}'' + \frac{6}{5}u_{2}^{(4)}u_{4}' - \frac{33}{25}u_{2}^{(5)}u_{3}' + \frac{1}{5}u_{2}^{(5)}u_{4} - \frac{7}{25}u_{2}^{(6)}u_{3} - \frac{1}{25}u_{2}^{(9)} + \frac{6}{25}u_{3}u_{3}'u_{4} + \frac{6}{25}u_{3}u_{3}'^2 + \frac{1}{25}u_{3}u_{3}^{(5)} + \frac{2}{5}u_{3}u_{4}^{(4)} - \frac{3}{5}u_{3}u_{5}''' + \frac{6}{25}u_{3}^2u_{3}'' + \frac{6}{5}u_{3}'u_{3}^{(4)} - \frac{3}{5}u_{3}'u_{5}'' + \frac{12}{5}u_{3}''u_{3}''' - \frac{2}{5}u_{3}''u_{5}' - \frac{1}{5}u_{3}^{(4)}u_{4} - \frac{1}{5}u_{3}^{(8)} + \frac{2}{5}u_{4}u_{4}''' - \frac{3}{5}u_{4}u_{5}'' - \frac{3}{5}u_{4}'u_{5}' + \frac{2}{5}u_{4}^{(7)} - \frac{6}{5}u_{5}u_{5}' - \frac{2}{5}u_{5}^{(6)}\\
	H_{6,1}(u_2,u_3,u_4,u_5) = & -\frac{132}{25}u_{2}u_{2}'u_{2}''' - \frac{18}{25}u_{2}u_{2}'u_{3}'' + \frac{39}{25}u_{2}u_{2}'u_{4}' - \frac{27}{25}u_{2}u_{2}''u_{3}' + \frac{3}{5}u_{2}u_{2}''u_{4} - \frac{96}{25}u_{2}u_{2}''^2 - \frac{21}{25}u_{2}u_{2}'''u_{3} - \frac{26}{25}u_{2}u_{2}^{(6)} + \frac{18}{25}u_{2}u_{3}u_{3}'' + \frac{6}{25}u_{2}u_{3}u_{4}' + \frac{6}{25}u_{2}u_{3}'u_{4} + \frac{18}{25}u_{2}u_{3}'^2 - \frac{14}{25}u_{2}u_{3}^{(5)} + \frac{9}{5}u_{2}u_{4}^{(4)} - \frac{9}{5}u_{2}u_{5}''' - \frac{24}{125}u_{2}^2u_{2}'u_{3} - \frac{72}{125}u_{2}^2u_{2}'^2 - \frac{24}{25}u_{2}^2u_{2}^{(4)} - \frac{3}{25}u_{2}^2u_{3}''' + \frac{3}{5}u_{2}^2u_{4}'' - \frac{6}{25}u_{2}^2u_{5}' - \frac{36}{125}u_{2}^3u_{2}'' - \frac{19}{25}u_{2}'u_{2}''u_{3} - \frac{88}{25}u_{2}'u_{2}^{(5)} - \frac{6}{25}u_{2}'u_{3}u_{3}' + \frac{18}{25}u_{2}'u_{3}u_{4} - \frac{12}{5}u_{2}'u_{3}^{(4)} + \frac{21}{5}u_{2}'u_{4}''' - \frac{9}{5}u_{2}'u_{5}'' - \frac{144}{25}u_{2}'^2u_{2}'' - \frac{3}{25}u_{2}'^2u_{4} - \frac{38}{5}u_{2}''u_{2}^{(4)} - \frac{6}{25}u_{2}''u_{3}^2 - 6u_{2}''u_{3}''' + \frac{34}{5}u_{2}''u_{4}'' - \frac{4}{5}u_{2}''u_{5}' - \frac{36}{5}u_{2}'''u_{3}'' + 5u_{2}'''u_{4}' - \frac{24}{5}u_{2}'''^2 - \frac{21}{5}u_{2}^{(4)}u_{3}' + u_{2}^{(4)}u_{4} - \frac{31}{25}u_{2}^{(5)}u_{3} - \frac{2}{5}u_{2}^{(8)} + \frac{6}{5}u_{3}u_{3}^{(4)} + \frac{1}{5}u_{3}u_{4}''' - \frac{6}{5}u_{3}u_{5}'' + \frac{12}{25}u_{3}^2u_{3}' + \frac{24}{5}u_{3}'u_{3}''' - \frac{3}{5}u_{3}'u_{4}'' - \frac{6}{5}u_{3}'u_{5}' - \frac{2}{5}u_{3}''u_{4}' + \frac{18}{5}u_{

In [162]:
print(_)

\left\{\begin{array}{rl}
	H_{6,0}(u_2,u_3,u_4,u_5) = & -\frac{7}{5}u_{2}u_{2}'u_{2}^{(4)} - \frac{3}{5}u_{2}u_{2}'u_{3}''' + \frac{18}{25}u_{2}u_{2}'u_{4}'' + \frac{3}{25}u_{2}u_{2}'u_{5}' - \frac{24}{125}u_{2}u_{2}'^2u_{3} - \frac{24}{125}u_{2}u_{2}'^3 - \frac{66}{25}u_{2}u_{2}''u_{2}''' - \frac{27}{25}u_{2}u_{2}''u_{3}'' + \frac{18}{25}u_{2}u_{2}''u_{4}' - \frac{21}{25}u_{2}u_{2}'''u_{3}' + \frac{3}{25}u_{2}u_{2}'''u_{4} - \frac{9}{25}u_{2}u_{2}^{(4)}u_{3} - \frac{4}{25}u_{2}u_{2}^{(7)} + \frac{3}{25}u_{2}u_{3}u_{3}''' + \frac{6}{25}u_{2}u_{3}u_{4}'' - \frac{6}{25}u_{2}u_{3}u_{5}' + \frac{18}{25}u_{2}u_{3}'u_{3}'' - \frac{3}{25}u_{2}u_{3}''u_{4} - \frac{8}{25}u_{2}u_{3}^{(6)} + \frac{6}{25}u_{2}u_{4}u_{4}' + \frac{16}{25}u_{2}u_{4}^{(5)} - \frac{3}{5}u_{2}u_{5}^{(4)} - \frac{72}{125}u_{2}^2u_{2}'u_{2}'' - \frac{12}{125}u_{2}^2u_{2}'u_{4} - \frac{12}{125}u_{2}^2u_{2}''u_{3} - \frac{27}{125}u_{2}^2u_{2}^{(5)} - \frac{3}{25}u_{2}^2u_{3}^{(4)} + \frac{6}{25}u_{2}^2u_{4}''' - \frac{3}{25}

In [163]:
order = lambda P : max([sum(o for o in el.orders() if o != -1) for el in P.monomials()])

In [164]:
for i in range(4):
    print(i, (order(H[6][i]), H[6][i].degree(), len(H[6][i].monomials())))

0 (5, 4, 77)
1 (5, 4, 60)
2 (4, 4, 35)
3 (3, 3, 24)


## Recursion operator